# Домашнее задание: рекомендательные системы - 1

В этом домашнем задании будем работать с данными о пользователях и оценках, которые они поставили различным фильмам.

На основе этих данных будем рекомендовать пользователям к просмотру новые для них фильмы.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from typing import List, Dict
from tqdm import tqdm, tqdm_notebook

## Загрузка и обработка данных

Загрузим данные.

In [2]:
ratings = pd.read_csv('https://raw.githubusercontent.com/aiedu-courses/stepik_applied_tasks/main/datasets/movies_ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp,title
0,1,31,2.5,1260759144,Dangerous Minds
1,7,31,3.0,851868750,Dangerous Minds
2,31,31,4.0,1273541953,Dangerous Minds
3,32,31,4.0,834828440,Dangerous Minds
4,36,31,3.0,847057202,Dangerous Minds


In [5]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings['userId'] = user_encoder.fit_transform(ratings['userId'])
ratings['movieId'] = item_encoder.fit_transform(ratings['movieId'])

num_users, num_movies = ratings.userId.nunique(), ratings.movieId.nunique()
num_users, num_movies

(671, 9025)

Поделим выборку на train и test так, чтобы у каждого пользователя последние 10 фильмов оказались в тесте для подсчета метрики качества рекомендаций k=10.  

In [7]:
ratings.groupby('userId')

In [8]:
train, test = [], []
num_test_samples = 10

for user, data in ratings.groupby('userId'):
    train += [data[:-num_test_samples]]
    test += [data[-num_test_samples:]]

train = pd.concat(train)
test = pd.concat(test)
print(train.shape, test.shape)

(93140, 5) (6710, 5)


In [26]:
train.head()

,userId,movieId,rating,timestamp,title
0,0,30,2.5,1260759144,Dangerous Minds
42,0,830,3.0,1260759179,Dumbo
84,0,856,3.0,1260759182,Sleepers
117,0,903,2.0,1260759185,Escape from New York
165,0,927,4.0,1260759205,Cinema Paradiso


## Quiz

Запишите данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

Назовите полученную таблицу `interactions`, действуйте по аналогии или воспользуйтесь кодом из урока.

В ответ запишите максимальное значение `movieId` из тестовых фильмов для пользователя `userId=2`.

In [94]:
interactions = (
    train
    .groupby('userId')['movieId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'movieId': 'true_train'})
    .set_index('userId')
)
    
interactions['true_test'] = (
    test
    .groupby('userId')['movieId'].agg(lambda x: list(x))
    
      
)

interactions.head()

,true_train,true_test
userId,,
0,"[30, 830, 856, 903, 927, 1013, 1037, 1043, 107...","[1107, 1136, 1511, 1661, 1704, 1739, 1811, 195..."
1,"[9, 16, 37, 45, 48, 49, 58, 100, 123, 129, 132...","[518, 519, 520, 521, 522, 523, 524, 525, 543, ..."
2,"[100, 266, 321, 341, 472, 521, 524, 525, 56, 2...","[5008, 5107, 5456, 5461, 5874, 6345, 6518, 656..."
3,"[1107, 1511, 1661, 1739, 2375, 9, 132, 163, 26...","[2491, 2495, 2543, 2575, 2576, 2602, 2606, 261..."
4,"[1811, 37, 129, 321, 328, 331, 341, 447, 519, ...","[5955, 5957, 6098, 6118, 6144, 6172, 6260, 627..."


In [97]:
max(interactions.loc[2, 'true_test'])

7681

Для оценки качества модели будем использовать метрику  precision@10 для каждого пользователя (доля угаданных рекомендаций). Усредним ее по всем пользователям (полученная метрика называется MAP@10).

In [98]:
def calc_precision(column):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

## Коллаборативная фильтрация

## Quiz

Составьте матрицу "оценок" пользователей - `ratings`. Нули будут обозначать отсутствие взаимодействия.

Действуйте по аналогии или воспользуйтесь кодом из урока.

В ответ запишите число столбцов в матрице `ratings`.

In [108]:
ratings = pd.pivot_table(
    train,
    values='rating',
    index='userId',
    columns='movieId').fillna(0)

In [110]:
ratings.shape[1]

8044

In [111]:
ratings_m = ratings.values

In [112]:
ratings_m

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

## Quiz

Посчитайте схожести пользователей (запишите их в np.array `similarity_users`) с помощью корреляции Пирсона. Для каждой пары учитывайте только ненулевые значения.

Действуйте по аналогии или воспользуйтесь кодом из урока.

В ответ запишите значение `similarity_users[0,6]` без округления.

In [113]:
import numpy as np
from tqdm.notebook import tqdm_notebook

# ratings_m - это матрица оценок пользователей, где строки представляют пользователей, а столбцы - фильмы.
# Нули могут обозначать отсутствие оценки.

# Создаем матрицу для хранения коэффициентов корреляции между пользователями
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

# Внешний цикл по всем пользователям
for i in tqdm_notebook(range(len(ratings_m)-1)):
    # Внутренний цикл начиная с пользователя, следующего после текущего пользователя (i+1)
    for j in range(i+1, len(ratings_m)):
        # Создаем булеву маску для общих ненулевых оценок между пользователями i и j
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)

        # Пропускаем, если нет общих ненулевых оценок
        if np.sum(mask_uv) == 0:
            continue

        # Выбираем ненулевые оценки для текущей пары пользователей
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        # Пропускаем, если у одного из пользователей или обоих пользователей есть только одна уникальная оценка
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue

        # Вычисляем коэффициент корреляции Пирсона между оценками пользователей i и j
        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        # Симметрично заполняем матрицу корреляции
        similarity_users[j,i] = similarity_users[i,j]


  0%|          | 0/670 [00:00<?, ?it/s]

In [116]:
similarity_users[0,6]


-0.4999999999999999

## Quiz

Сделайте user-based прогнозы по тому же правилу, что и в уроке:

Для каждого пользователя:

1. Найдём пользователей с похожестью больше $\alpha$ на нашего пользователя.
2. Посчитаем для каждого фильма долю пользователей (среди выделенных на первом шаге), которые взаимодействовали с этим фильмом.
3. Порекомендуем фильмы с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

В нашем примере данных не очень много, поэтому возьмём $\alpha = 0$.

Сделайте предсказания и запишите их в столбец
`prediction_user_based` таблицы `interactions`.

В ответ запишите минимальный предсказанный `movieId` для пользователя `userId=4`.

In [117]:
# prediction_user_based - это список, в который будут добавляться рекомендации для каждого пользователя
prediction_user_based = []

# Внешний цикл по всем пользователям
for i in tqdm_notebook(range(len(similarity_users))):
    # Выделяем пользователей, у которых есть положительная корреляция с текущим пользователем
    users_sim = similarity_users[i] > 0
    
    # Если нет пользователей с положительной корреляцией, добавляем пустой список в prediction_user_based
    if len(users_sim) == 0:
        prediction_user_based.append([])
    else:
        # Сортируем фильмы по сумме оценок пользователей с положительной корреляцией в убывающем порядке
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        
        # Преобразуем индексы фильмов в названия фильмов
        tmp_recommend = ratings.columns[tmp_recommend]
        
        # Исключаем фильмы, которые уже оценены текущим пользователем, и выбираем топ-10 рекомендаций
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i])][:10]
        
        # Добавляем рекомендации для текущего пользователя в prediction_user_based
        prediction_user_based.append(list(recommend))

# Добавляем столбец 'prediction_user_based' к датасету interactions
interactions['prediction_user_based'] = prediction_user_based


  0%|          | 0/671 [00:00<?, ?it/s]

In [120]:
min(interactions.loc[4, 'prediction_user_based'])

100

## Quiz

Посчитайте значение метрики MAP@10 для user-based подхода.

Ответ округлите до тысячных.

In [121]:
calc_precision('prediction_user_based')

0.005365126676602086

## SVD-разложение

Для выполнения заданий при необходимости заглядывайте в ноутбук из урока.

## Quiz

Сделайте сингулярное разложение (svd в scipy.linalg), на выходе вы получите три матрицы - `U`,`sigma`,`V`.

В ответ запишите число элементов матрицы `U`.

In [123]:
from scipy.linalg import svd

In [124]:
U, sigma, V = svd(ratings)
print(ratings.shape, U.shape, sigma.shape, V.shape)

(671, 8044) (671, 671) (671,) (8044, 8044)


In [126]:
Sigma = np.zeros((671, 8044))
Sigma[:671, :671] = np.diag(sigma)

new_ratings = U.dot(Sigma).dot(V)

print(sum(sum((new_ratings - ratings.values) ** 2)))

3.4332441395842543e-22


Значения у матрицы с сингулярными числами отсортированы по убыванию.

Оставьте только первые 150 компонент, чтобы получить скрытые представления размерности 150. Для этого необходимо оставить 150 столбцов в матрице U, оставить из sigma только первые 150 значений (и сделать из них диагональную матрицу) и 150 столбцов в матрице V. Перемножим преобразованные матрицы ($\hat{U}, \hat{sigma}, \hat{V^T}$), чтобы получить восстановленную матрицу оценок.

In [127]:
K = 150

sigma[K:] = 0
Sigma = np.zeros((671, 8044))
Sigma[:671, :671] = np.diag(sigma)

## Quiz

Посчитайте качество аппроксимации матрицы по норме Фробениуса (среднеквадратичную ошибку между всеми элементами соответствующими элементами двух матриц). Сравните его с простым бейзлайном с константным значением, равным среднему значению исходной матрицы. У аппроксимации ошибка должна получиться ниже.

Во сколько раз ошибка аппроксимации меньше, чем ошибка бейзлайна? Ответ округлите до целого числа.

In [129]:
new_ratings = U.dot(Sigma).dot(V)

print(sum(sum((new_ratings - ratings.values) ** 2))) # апрроксимация
print(sum(sum((ratings.values.mean() - ratings.values) ** 2))) #base line

248017.78364373572
1255671.6721978304


## Quiz

Теперь сделайте предсказания по матрице (не забывайте про то, что уже было просмотрено пользователем).

Для этого необходимо для каждого пользователя найти предметы с наибольшими оценками в восстановленной матрице.

В ответ запишите максимальный предсказанный `movieId` для пользователя `userId=4`.

In [130]:
top_k = 10

new_ratings = pd.DataFrame(new_ratings, index=ratings.index, columns=ratings.columns)

predictions = []

for personId in tqdm_notebook(interactions.index):
    prediction = (
        new_ratings
        .loc[personId]
        .sort_values(ascending=False)
        .index.values
    )

    predictions.append(
        list(prediction[~np.in1d(
            prediction,
            interactions.loc[personId, 'true_train'])])[:top_k])

interactions['prediction_svd'] = predictions

  0%|          | 0/671 [00:00<?, ?it/s]

In [135]:
max(interactions.loc[4, 'prediction_svd'])

3373

## Quiz

Посчитайте значение метрики MAP@10 для SVD-подхода.

Ответ округлите до тысячных.

In [131]:
calc_precision('prediction_svd')

0.022652757078986587